In [1]:
from pathlib import Path
path = Path('/data/DIGITANIE_v4/normalisation_stats.npy')
import numpy as np
stats = np.load(path, allow_pickle=True).item()

In [2]:
stats['ARCACHON'.title()]

{'mean': array([399.75224581, 568.22127573, 513.3289946 , 794.80494907]),
 'std': array([ 507.90247899,  420.33246958,  329.08664137, 1094.20216753]),
 'perc_2': array([117., 248., 243.,  84.]),
 'perc_98': array([2355., 2020., 1599., 3778.]),
 'mean_clip': array([384.33698218, 554.41821743, 503.13296748, 785.34202384]),
 'std_clip': array([ 419.96037018,  339.06449895,  264.49784969, 1062.69335525]),
 'perc_995': array([3505., 2987., 2230., 4379.]),
 'mean_clip995': array([397.54310305, 565.20022728, 509.85630767, 792.07991096]),
 'std_clip995': array([ 491.13929391,  395.69034571,  300.04755905, 1083.31749887])}

In [ ]:
c = 'BUENOS-AIRES'.title()

In [32]:
stats.keys()

dict_keys(['Amsterdam', 'Arcachon', 'Atlanta', 'Barcelone', 'Berlin', 'Biarritz', 'Bordeaux', 'Boyoali', 'Bucarest', 'Bunia', 'Copenhague', 'Dar-Es-Salaam', 'Duck', 'Dusseldorf', 'Eindhoven', 'Gaborone', 'Helena', 'Helsinki', 'Ile-De-France', 'Ingolstadt', 'Karlsruhe', 'Kinshasa', 'Leeds', 'Maros', 'Montpellier', 'Moscou', 'Munich', 'Nantes', 'New-York', 'Paris', 'Samarkand', 'San-Francisco', 'Strasbourg', 'Toulouse', 'Milan', 'Brisbane', 'Buenos-Aires', 'Can-Tho', 'Lagos', 'Le-Caire', 'Port-Elisabeth', 'Rio-Janeiro', 'Shanghai', 'Tianjin'])

In [31]:
a= torch.randint(high=100, size=(3,10,10))
a = torch.clamp(a, max=torch.Tensor([10, 50, 90]).reshape((-1,1,1)))
a -= torch.Tensor([2, 0, 1000]).reshape((-1,1,1))
a /= torch.Tensor([8, 45, 900]).reshape((-1,1,1))
a

tensor([[[ 0.1250,  1.0000,  1.0000,  1.0000,  0.5000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000],
         [ 0.5000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000, -0.1250,  1.0000],
         [ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000],
         [ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  0.7500,
           1.0000,  1.0000,  1.0000],
         [ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000],
         [ 1.0000,  1.0000,  0.1250,  1.0000,  1.0000,  1.0000,  0.3750,
           1.0000,  0.7500,  1.0000],
         [ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000],
         [ 1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000],
         [ 1.0000,  1.0000, -0.1250,  1.0000,  1.0000,  1.0000,  1.0000,
           1.0000,  1.0000,  1.0000],
 